<a href="https://colab.research.google.com/github/Dominic-byte/Team-13-Regression-Predict/blob/master/Sendy_regressor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Team 13 Sendy Logistics Challenge

## Contents

1. Introduction
2. Imports
3. Definitions
4. Data cleaning and preprocessing
5. Exploratoory data analysis
6. Feature engineering and section
7. Model building
8. Results interpretation
9. Conclusion

## 2. Imports

In [0]:
%%capture
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
%matplotlib inline
pd.options.display.max_columns = 60

In [143]:
train_df = pd.read_csv('https://raw.githubusercontent.com/Dominic-byte/Team-13-Regression-Predict/master/Train.csv', index_col=0)
test_df = pd.read_csv('https://raw.githubusercontent.com/Dominic-byte/Team-13-Regression-Predict/master/Test.csv', index_col=0)
riders_df = pd.read_csv('https://raw.githubusercontent.com/Dominic-byte/Team-13-Regression-Predict/master/Riders.csv', index_col=0)
variables = pd.read_csv('https://raw.githubusercontent.com/Dominic-byte/Team-13-Regression-Predict/master/VariableDefinitions.csv', index_col=0)
print(train_df.shape, test_df.shape, riders_df.shape)
train_df.head(3)

(21201, 28) (7068, 24) (960, 4)


,User Id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),Confirmation - Time,Arrival at Pickup - Day of Month,Arrival at Pickup - Weekday (Mo = 1),Arrival at Pickup - Time,Pickup - Day of Month,Pickup - Weekday (Mo = 1),Pickup - Time,Arrival at Destination - Day of Month,Arrival at Destination - Weekday (Mo = 1),Arrival at Destination - Time,Distance (KM),Temperature,Precipitation in millimeters,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id,Time from Pickup to Arrival
Order No,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Order_No_4211,User_Id_633,Bike,3,Business,9,5,9:35:46 AM,9,5,9:40:10 AM,9,5,10:04:47 AM,9,5,10:27:30 AM,9,5,10:39:55 AM,4,20.4,NaN,-1.317755,36.830370,-1.300406,36.829741,Rider_Id_432,745
Order_No_25375,User_Id_2285,Bike,3,Personal,12,5,11:16:16 AM,12,5,11:23:21 AM,12,5,11:40:22 AM,12,5,11:44:09 AM,12,5,12:17:22 PM,16,26.4,NaN,-1.351453,36.899315,-1.295004,36.814358,Rider_Id_856,1993
Order_No_1899,User_Id_265,Bike,3,Business,30,2,12:39:25 PM,30,2,12:42:44 PM,30,2,12:49:34 PM,30,2,12:53:03 PM,30,2,1:00:38 PM,3,NaN,NaN,-1.308284,36.843419,-1.300921,36.828195,Rider_Id_155,455


## 3. Definitions

In [144]:
pd.options.display.max_colwidth = 0
variables

,Unique number identifying the order
Order No,
User Id,Unique number identifying the customer on a platform
Vehicle Type,"For this competition limited to bikes, however in practice Sendy service extends to trucks and vans"
Platform Type,"Platform used to place the order, there are 4 types"
Personal or Business,Customer type
Placement - Day of Month,Placement - Day of Month i.e 1-31
Placement - Weekday (Mo = 1),Placement - Weekday (Monday = 1)
Placement - Time,Placement - Time - Time of day the order was placed
Confirmation - Day of Month,Confirmation - Day of Month i.e 1-31
Confirmation - Weekday (Mo = 1),Confirmation - Weekday (Monday = 1)


In [0]:
pd.options.display.max_colwidth = 50

## 4. Data cleaning and preprocessing

In [146]:
pd.merge(train_df, riders_df, how='left', on='Rider Id').info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21201 entries, 0 to 21200
Data columns (total 32 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   User Id                                    21201 non-null  object 
 1   Vehicle Type                               21201 non-null  object 
 2   Platform Type                              21201 non-null  int64  
 3   Personal or Business                       21201 non-null  object 
 4   Placement - Day of Month                   21201 non-null  int64  
 5   Placement - Weekday (Mo = 1)               21201 non-null  int64  
 6   Placement - Time                           21201 non-null  object 
 7   Confirmation - Day of Month                21201 non-null  int64  
 8   Confirmation - Weekday (Mo = 1)            21201 non-null  int64  
 9   Confirmation - Time                        21201 non-null  object 
 10  Arrival at Pickup - Da

In [147]:
riders_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 960 entries, Rider_Id_396 to Rider_Id_904
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   No_Of_Orders    960 non-null    int64  
 1   Age             960 non-null    int64  
 2   Average_Rating  960 non-null    float64
 3   No_of_Ratings   960 non-null    int64  
dtypes: float64(1), int64(3)
memory usage: 37.5+ KB


### Data types
#### **Numerical**
- Ratio
  - *Number of Orders*
  - *Age*
  - *Number of Ratings*
  - *Time from pickup to arrival*
- Interval
  - *Time (Placement, Confimation, Arrival, etc)* 
  - *Distance (KM)*
  - *Temperature*
  - *Precipitation
  - *Co-ordinates*

#### **Categorical**
- Nominal
  - *User Id*
    - This is categorical as it only varies in the user that has\
      placed the order.
    - It may be possible to determine the frequency of orders\
      placed by customers and determine their loyalty. This may\
      reflect a drivers motivation to provide higher quality\
      service to that user.
  - *Vehicle Type*
    - The type of vehicle used would ordinarily have an impact on\
      delivery time as a motorcycle would be faster than a truck.
    - Since there is only one vehicle type to consider (bike)\
      there is no value to be gained from this data
  - *Platform Type*
    - The source of order placement is qualitative and may give\
      information about the level of service delivery for the\
      various platforms. More frequently used platforms may be\
      serviced better by drivers as they may provide a steady\
      stream of customers.
    - The platform can only take on the values from 1 to 4 and\
      should be stored as an `int16`
  - *Personal or Business*
    - Business orders are likely to be frequent and and located\
      near central business district, whereas personal orders\
      may require further travel to suburban areas. This could\
      have an effect on the travel time.
  - *Day of month (Placement, Confimation, Arrival, etc)*
    - The day of month can only vary in its quality. There may\
      more or less orders placed on a given day of the month.\
    - Since the values of day of month can take on values between\
      1 and 31 it would be most effecient to store it as an `int16`\
      instead of `int64` which is for larger numbers.
    - It may be that more orders are placed on certain days that\
      would result in longer delivery times.    
  - *Weekday*
    - Businesses typically operate on weekdays. It would logically\
      follow that weekdays are busier than weekends resulting in\
      longer delivery times due  to traffic or the volume of\
      orders.
    - Since the weekday is represented by 1 to 7, it should be\
      stored as an `int16`
    - *Rider Id*

- Ordinal
  - *Average rating"

### Cleaning functions

In [148]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21201 entries, Order_No_4211 to Order_No_9836
Data columns (total 28 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   User Id                                    21201 non-null  object 
 1   Vehicle Type                               21201 non-null  object 
 2   Platform Type                              21201 non-null  int64  
 3   Personal or Business                       21201 non-null  object 
 4   Placement - Day of Month                   21201 non-null  int64  
 5   Placement - Weekday (Mo = 1)               21201 non-null  int64  
 6   Placement - Time                           21201 non-null  object 
 7   Confirmation - Day of Month                21201 non-null  int64  
 8   Confirmation - Weekday (Mo = 1)            21201 non-null  int64  
 9   Confirmation - Time                        21201 non-null  object 
 10  Arrival

In [149]:
train_df['Temperature'].isnull().sum()/len(train_df)

0.20593368237347295

In [0]:
def data_preprocessing(df, na_thresh = 0, strategy = 'mean'):
  """Function that preprocesses data used for predictions and testing
  Parameters
  ----------
  df: DataFrame
          DataFrame to preprocess.
  na_thresh: float
          float between 0.0 and 1.0 specifying the drop/fillna
          threshhold
  strategy: string
          Filling strategy, 'mean', 'median' or  'rolling'
  Returns
  -------
  df: DataFrame
          Preprocessed DataFrame

  """
  cdf = df.copy()
  old = cdf.memory_usage().sum()
  # Create lists of columns to change
  integers = [col for col in cdf.columns if cdf[col].dtypes == 'int64']
  floats = [col for col in cdf.columns if cdf[col].dtypes == 'float64']
  time_cols = [col for col in cdf.columns if col.endswith("Time")]

  # Reduce size of data storage types
  cdf[integers] = cdf[integers].astype('int16')
  cdf[floats] = cdf[floats].astype('float16')
  for col in time_cols:
    cdf[col] = pd.to_datetime(cdf[col]).dt.time
  
  # Correcting specific columns
  if 'Distance (KM)' in cdf.columns:
    cdf['Distance (KM)'] = cdf['Distance (KM)'].astype('float16') 
  
  # Dropping NaN's
  for col in cdf.columns:
    missing = cdf[col].isnull().sum()/len(cdf_df)
    if missing > na_thresh:
      cdf.drop(columns=col, inplace=True)
  
  # Filling NaN's
  for col in cdf.columns:
    if cdf[col].isnull().sum() > 0:
      if cdf[col].dtypes == 'object':
        cdf[col].fillna(cdf[col].mode(), inplace=True) 
      elif strategy == 'mean':
        cdf[col].fillna(round(cdf[col].mean()), inplace=True)
      elif strategy == 'median':
        cdf[col].fillna(round(cdf[col].median()), inplace=True)
      elif strategy == 'rolling':
        cdf[col].fillna(cdf[col].rolling(7).mean(), inplace=True)
      else:
        raise ValueError

  new = cdf.memory_usage().sum()
  print("Bytes\t","\nOld:\t", str(old), "\nNew:\t", "("+str(new)+")", "\nSaved:\t", str(old - new))
  return cdf

In [177]:
train1 = data_preprocessing(train_df, 1.0)
test1 = data_preprocessing(test_df, 1.0)
rider1 = data_preprocessing(riders_df, 1.0)

Bytes	 
Old:	 4918632 
New:	 (2501718) 
Saved:	 2416914
Bytes	 
Old:	 1413600 
New:	 (735072) 
Saved:	 678528


KeyError: ignored

In [176]:
new.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21201 entries, Order_No_4211 to Order_No_9836
Data columns (total 28 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   User Id                                    21201 non-null  object 
 1   Vehicle Type                               21201 non-null  object 
 2   Platform Type                              21201 non-null  int16  
 3   Personal or Business                       21201 non-null  object 
 4   Placement - Day of Month                   21201 non-null  int16  
 5   Placement - Weekday (Mo = 1)               21201 non-null  int16  
 6   Placement - Time                           21201 non-null  object 
 7   Confirmation - Day of Month                21201 non-null  int16  
 8   Confirmation - Weekday (Mo = 1)            21201 non-null  int16  
 9   Confirmation - Time                        21201 non-null  object 
 10  Arrival

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
True
False
False
False
False
False
False
False
